In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import sympy 
import math
from sympy import *
import copy
import warnings
warnings.filterwarnings("ignore")

In [2]:
bond_data = pd.read_excel("T_20210129.xlsx")

In [3]:
bond_data["today"] = dt.datetime.strptime("2021-01-29","%Y-%m-%d")

In [4]:
bond_data["date_diff"] = bond_data["Maturity"] - bond_data["today"]
bond_data["date_diff"] = bond_data["date_diff"].apply(lambda x:x.days)

In [5]:
def solve(number):

    y = sympy.symbols("y")
    total_sum = 0
    n = bond_data.loc[number,"date_diff"]//180
    to_next_coupon = bond_data.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += bond_data.loc[number,"Coupon"]/2*pow((1+y),-(each + to_next_coupon/180))
    total_sum += (100+bond_data.loc[number,"Coupon"]/2)*pow((1+y),-(n+to_next_coupon/180))
    # using
    f = bond_data.loc[number,"Ask Price"] - total_sum
    t = 0.00000001
    f1 = f.subs(y, t) # 表示t赋值给x
    while abs(f1) > 0.00000001:  #我们设定的精度
        dify = diff(f, y) # f对x求导
        dify = dify.subs(y, t)
        t = t - f1/dify
        f1 = f.subs(y, t)
    
    #print('y=', t)
    return t
    


In [6]:
for i in range(len(bond_data)):
    bond_data.loc[i,"half_YTM"] = solve(i)
    bond_data.loc[i,"YTM"] = 2 * solve(i)

In [7]:
def solve_duration(number):
    y = bond_data.loc[number,"half_YTM"]
    n = bond_data.loc[number,"date_diff"]//180
    total_sum = 0
    to_next_coupon = bond_data.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += (each/2+to_next_coupon/180) * bond_data.loc[number,"Coupon"]/2*pow((1+y),-(each + to_next_coupon/180))/ bond_data.loc[number,"Ask Price"]
    total_sum += ((100+bond_data.loc[number,"Coupon"]/2)*pow((1+y),-(n+to_next_coupon/180)))*(bond_data.loc[number,"date_diff"]/365) / bond_data.loc[number,"Ask Price"]
    return total_sum
    print(total_sum)


In [8]:
for i in range(len(bond_data)):
    
    bond_data.loc[i,"Duration"] = solve_duration(i)

In [9]:
def solve_PVBP(number):
    total_sum = 0
    y = bond_data.loc[number,"half_YTM"] + 0.00005
    n = bond_data.loc[number,"date_diff"]//180
    to_next_coupon = bond_data.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += bond_data.loc[number,"Coupon"]/2*pow((1+y),-(each + to_next_coupon/180))
    total_sum += (100+bond_data.loc[number,"Coupon"]/2)*pow((1+y),-(n+to_next_coupon/180))
    PVBP = total_sum * bond_data.loc[number,"Duration"]* 0.0001 / (1+bond_data.loc[number,"half_YTM"])
    return [PVBP,total_sum]

In [10]:
def compute_MD(number):
    #result = solve_PVBP(number)[0] * 10000 / solve_PVBP(number)[1]
    result = bond_data.loc[i,"Duration"]/(1+bond_data.loc[i,"half_YTM"])
    return result

In [11]:
for i in range(len(bond_data)):
    bond_data.loc[i,"MD"] = compute_MD(i)

In [12]:
def compute_convexity(number):
    y = bond_data.loc[number,"half_YTM"]
    n = bond_data.loc[number,"date_diff"]//180
    total_sum = 0
    to_next_coupon = bond_data.loc[number,"date_diff"]%(180)
    for each in range(n):
            total_sum += bond_data.loc[number,"Coupon"]/2*math.pow((1+y),-(each + to_next_coupon/180))*((each + to_next_coupon/180)+math.pow((each + to_next_coupon/180),2))
    total_sum += (100+bond_data.loc[number,"Coupon"]/2)*math.pow((1+y),-(n + to_next_coupon/180))*((n + to_next_coupon/180)+math.pow((n + to_next_coupon/180),2))
    total_sum = total_sum * 1/(bond_data.loc[number,"Ask Price"]*(math.pow((1+y),2)))
        #return total_sum
    #print(total_sum/4)
    return total_sum/4

    
    
    

In [13]:
for i in range(len(bond_data)):
    bond_data.loc[i,"Convexity"] = compute_convexity(i)

# For Each of these Bonds calculate Bond YTM, Duration, Modified Duration, Convexity

In [14]:
bond_data.head()

,Name,Ticker,Coupon,Maturity,Maturity Type,Country,Currency,Identifier,Bloomberg Composite Rating,Ask Price,Source,today,date_diff,half_YTM,YTM,Duration,MD,Convexity
0,United States Treasury Note/Bond,T,1.375,2021-01-31,NORMAL,US,USD,912828N89,NR,100.019531,BGN,2021-01-29,2,0.820400062152708,1.64080012430542,0.00547945205482591,0.00301002629517942,0.000848
1,United States Treasury Note/Bond,T,2.125,2021-01-31,NORMAL,US,USD,912828B58,NR,100.023438,BGN,2021-01-29,2,1.53479899749063,3.06959799498127,0.00547945205479452,0.00216169095072982,0.000437
2,United States Treasury Note/Bond,T,2.500,2021-01-31,NORMAL,US,USD,9128285X4,NR,100.031250,BGN,2021-01-29,2,1.97399507936561,3.94799015873123,0.00547945205479454,0.00184245498347071,0.000318
3,United States Treasury Note/Bond,T,2.250,2021-02-15,NORMAL,US,USD,9128283X6,NR,100.097656,BGN,2021-01-29,17,0.114178720654214,0.228357441308429,0.0465753424666503,0.0418023981281051,0.020816
4,United States Treasury Note/Bond,T,3.625,2021-02-15,NORMAL,US,USD,912828PX2,NR,100.152344,BGN,2021-01-29,17,0.190144885863182,0.380289771726365,0.0465753424657534,0.0391341785516925,0.018244


In [15]:
def solve_appr_conv(number):
    total_sum_plus = 0
    total_sum_minus = 0
    y_plus = bond_data.loc[number,"half_YTM"] + 0.00005
    y_minus = bond_data.loc[number,"half_YTM"] - 0.00005
    n = bond_data.loc[number,"date_diff"]//180
    to_next_coupon = bond_data.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum_plus += bond_data.loc[number,"Coupon"]/2*pow((1+y_plus),-(each + to_next_coupon/180))
    total_sum_plus += (100+bond_data.loc[number,"Coupon"]/2)*pow((1+y_plus),-(n+to_next_coupon/180))
    for each in range(n):
        total_sum_minus += bond_data.loc[number,"Coupon"]/2*pow((1+y_minus),-(each + to_next_coupon/180))
    total_sum_minus += (100+bond_data.loc[number,"Coupon"]/2)*pow((1+y_minus),-(n+to_next_coupon/180))
    a = (total_sum_minus+total_sum_plus - 2*bond_data.loc[number,"Ask Price"])/(0.0001**2*bond_data.loc[number,"Ask Price"])
    PVBP = total_sum * bond_data.loc[number,"Duration"]* 0.0001 / (1+bond_data.loc[number,"half_YTM"])
    return a


In [16]:
y = sympy.symbols("y")
total_sum = 0
for number in (bond_data[bond_data["date_diff"]>365].index):
    #print(number)
    n = bond_data.loc[number,"date_diff"]//180
    to_next_coupon = bond_data.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += bond_data.loc[number,"Coupon"]/2*pow((1+y),-(each + to_next_coupon/180))
    total_sum += (100+bond_data.loc[number,"Coupon"]/2)*pow((1+y),-(n+to_next_coupon/180))
    # using


    

In [17]:
f = bond_data[bond_data["date_diff"]>365]["Ask Price"].sum() - total_sum
t = 0.00001
f1 = f.subs(y, t) # 表示t赋值给x
while abs(f1) > 0.00001:  #我们设定的精度
    dify = diff(f, y) # f对x求导
    dify = dify.subs(y, t)
    t = t - f1/dify
    f1 = f.subs(y, t)
Portfolio_YTM = t    


In [18]:
test_bond = bond_data[bond_data["date_diff"]>365]
Portfolio_price = test_bond["Ask Price"].sum()
Portfolio_accrued_price = Portfolio_price + np.nansum(test_bond["Coupon"]/2*((180 - bond_data["date_diff"]%180)/180))
print("Por0 value included accrued interest is",Portfolio_accrued_price)

Por0 value included accrued interest is 29556.05034852222


In [19]:
y = Portfolio_YTM + 0.00005
total_sum = 0
for number in (bond_data[bond_data["date_diff"]>365].index):
    n = bond_data.loc[number,"date_diff"]//180
    to_next_coupon = bond_data.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += bond_data.loc[number,"Coupon"]/2*pow((1+y),-(each + to_next_coupon/180))
    total_sum += (100+bond_data.loc[number,"Coupon"]/2)*pow((1+y),-(n+to_next_coupon/180))

new_Portfolio_price = total_sum
PVBP = Portfolio_price - new_Portfolio_price

In [20]:
Appr_duration = PVBP * (1+Portfolio_YTM) / new_Portfolio_price * 10000


In [21]:
portfolio_duration = sum(test_bond["Ask Price"]/test_bond["Ask Price"].sum() * test_bond["Duration"])


In [22]:
Modified_duration = portfolio_duration / (1 + Portfolio_YTM)


In [23]:
YTM_plus = Portfolio_YTM + 0.00005
YTM_minus = Portfolio_YTM - 0.00005
price_plus = 0
price_minus = 0
for number in (bond_data[bond_data["date_diff"]>365].index):
    n = bond_data.loc[number,"date_diff"]//180
    to_next_coupon = bond_data.loc[number,"date_diff"]%180
    for each in range(n):
        price_plus += bond_data.loc[number,"Coupon"]/2*pow((1+YTM_plus),-(each + to_next_coupon/180))
    price_plus += (100+bond_data.loc[number,"Coupon"]/2)*pow((1+YTM_plus),-(n+to_next_coupon/180))

for number in (bond_data[bond_data["date_diff"]>365].index):
    n = bond_data.loc[number,"date_diff"]//180
    to_next_coupon = bond_data.loc[number,"date_diff"]%180
    for each in range(n):
        price_minus += bond_data.loc[number,"Coupon"]/2*pow((1+YTM_minus),-(each + to_next_coupon/180))
    price_minus += (100+bond_data.loc[number,"Coupon"]/2)*pow((1+YTM_minus),-(n+to_next_coupon/180))
    

In [24]:
Appr_convexity = (price_plus + price_minus - 2 * Portfolio_price) / (pow(0.0001,2) * Portfolio_price)



In [25]:
y = Portfolio_YTM
total_sum = 0
for number in (bond_data[bond_data["date_diff"]>365].index):
    n = bond_data.loc[number,"date_diff"]//180
    to_next_coupon = bond_data.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += bond_data.loc[number,"Coupon"]*math.pow((1+y),-(each + to_next_coupon/180))*((each + to_next_coupon/180)+math.pow((each + to_next_coupon/180),2))
    total_sum += (100+bond_data.loc[number,"Coupon"])*math.pow((1+y),-(n + to_next_coupon/180))*((n + to_next_coupon/180)+math.pow((n + to_next_coupon/180),2))
total_sum = total_sum * 1/(Portfolio_price*(math.pow((1+y),2)))/4



# Assume you are holding a portfolio (por0) which consist of one UNIT of each of the above bonds with more than 1 year to maturity. For por0 Calculate :
Dollar Value of the portfolio,Portfolio YTM,Actual and Approximate Duration,Actual Modified Duration,Actual and Approximate Convexity

In [26]:
print("Dollar value of the portfolio is",PVBP)
print('Portfolio annualized YTM is', 2*t)
print("Actual portfolio duration is",portfolio_duration)
print("Approximate portfolio duration is",Appr_duration)
print("Actual modified duration is",Modified_duration)
print("Approximate portfolio convexity is",Appr_convexity)
print("Actual portfolio convexity is",total_sum)

Dollar value of the portfolio is 21.4099322571274
Portfolio annualized YTM is 0.0128947057410030
Actual portfolio duration is 6.96683410980636
Approximate portfolio duration is 7.33585454427263
Actual modified duration is 6.92220421658039
Approximate portfolio convexity is 114.437490309645
Actual portfolio convexity is 136.1938295217289


# Assume You are a Long Duration Fixed Income Portfolio Manager and you have $100,000,000 to invest in the above Government Bonds with the following Investment Mandate



You can only invest in Long term maturity bonds (bonds with maturity greater than 17 years)


Maximum dollar exposure to each particular bond should be less than 10% of the overall value of your portfolio.


You can only hold long exposure.

In [27]:
fund = 100000000
bond_bank = bond_data[bond_data["date_diff"]>365*17]
#bond_bank["YTM"] = bond_bank["YTM"].apply(lambda x:float(x))
bond_bank = bond_bank.sort_values(by='Duration',ascending = False)


In [28]:
# Since I believe long term rates are going to decline in the next year. 
# The price of bond with higher convexity would increase higher than others.
# Then bonds with higher convexity will be beneficial in this situation
# So I add bonds with Convexity from higher to lower, to satisfy the exposure requirement
# And invest each of them with equal weight

In [29]:
por1 = bond_bank.iloc[[0,1],:]
for i,index in enumerate(bond_bank.index):
    if i >= 2:
        price_list = list(por1["Ask Price"])
        price_list.append(bond_bank.loc[index,"Ask Price"])
        total_value = sum(price_list)
        por1 = por1.append(bond_bank.loc[index,:])
        for price in (price_list):
            mark = 0
            if price/total_value > 0.1:     
                break
            else:
                mark = 1

        if mark == 1:
            break
por1   

,Name,Ticker,Coupon,Maturity,Maturity Type,Country,Currency,Identifier,Bloomberg Composite Rating,Ask Price,Source,today,date_diff,half_YTM,YTM,Duration,MD,Convexity
317,United States Treasury Note/Bond,T,1.375,2050-08-15,NORMAL,US,USD,912810SP4,NR,89.593750,BGN,2021-01-29,10790,0.00914613767301540,0.0182922753460308,24.1783524006396,23.9592180934194,674.694688
316,United States Treasury Note/Bond,T,1.250,2050-05-15,NORMAL,US,USD,912810SN9,NR,86.765625,BGN,2021-01-29,10698,0.00923068921338427,0.0184613784267685,24.1550127324112,23.9340846355337,675.163444
318,United States Treasury Note/Bond,T,1.625,2050-11-15,NORMAL,US,USD,912810SS8,NR,95.328125,BGN,2021-01-29,10882,0.00923248340768087,0.0184649668153617,23.5296455566811,23.3143957844411,655.918167
315,United States Treasury Note/Bond,T,2.000,2050-02-15,NORMAL,US,USD,912810SL3,NR,104.250000,BGN,2021-01-29,10609,0.00907904557557365,0.0181580911511473,22.5372380757951,22.3344624730959,602.130239
313,United States Treasury Note/Bond,T,2.250,2049-08-15,NORMAL,US,USD,912810SJ8,NR,109.968750,BGN,2021-01-29,10425,0.00905155475663040,0.0181031095132608,21.8109013527185,21.6152497361534,568.210361
314,United States Treasury Note/Bond,T,2.375,2049-11-15,NORMAL,US,USD,912810SK5,NR,112.890625,BGN,2021-01-29,10517,0.00916606221887355,0.0183321244377471,21.5092167349888,21.3138526356070,564.525208
312,United States Treasury Note/Bond,T,2.875,2049-05-15,NORMAL,US,USD,912810SH2,NR,124.203125,BGN,2021-01-29,10333,0.00913058748533187,0.0182611749706637,20.5136713317881,20.3280641635355,521.251444
311,United States Treasury Note/Bond,T,3.000,2049-02-15,NORMAL,US,USD,912810SF6,NR,126.906250,BGN,2021-01-29,10244,0.00897209985377410,0.0179441997075482,20.5068519570256,20.3244985267656,512.292870
309,United States Treasury Note/Bond,T,3.000,2048-08-15,NORMAL,US,USD,912810SD1,NR,126.593750,BGN,2021-01-29,10060,0.00896717235104877,0.0179343447020975,20.2267509621796,20.0469861819668,497.623767
301,United States Treasury Note/Bond,T,2.250,2046-08-15,NORMAL,US,USD,912810RT7,NR,109.875000,BGN,2021-01-29,9329,0.00890935812872161,0.0178187162574432,20.0351939424221,19.8582695075625,474.154414


In [30]:
por1["Ask Price"]/por1["Ask Price"].sum()

317    0.067188
316    0.065068
318    0.071489
315    0.078180
313    0.082468
314    0.084659
312    0.093143
311    0.095170
309    0.094936
301    0.082398
305    0.090518
307    0.094783
Name: Ask Price, dtype: float64

In [31]:
# I will invest each bond equally in por1 portfolio

In [32]:
por1_accrued_price = por1["Ask Price"].sum() + np.nansum(por1["Coupon"]/2*((180 - por1["date_diff"]%180)/180))
print("por1 price with arrued interest is",por1_accrued_price)

por1 price with arrued interest is 1336.8680555555557


In [33]:
# number of por1 I will invest
number_invest = floor(fund / (por1_accrued_price))


In [34]:
money_left = fund - por1_accrued_price*number_invest


In [35]:
y = sympy.symbols("y")
total_sum = 0
for number in (por1.index):
    n = por1.loc[number,"date_diff"]//180
    to_next_coupon = por1.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += por1.loc[number,"Coupon"]/2*pow((1+y),-(each + to_next_coupon/180))*number_invest
    total_sum += (100+por1.loc[number,"Coupon"]/2)*pow((1+y),-(n+to_next_coupon/180))*number_invest
    # using
f = (por1["Ask Price"].sum()*number_invest) - total_sum
t = 0.0001
f1 = f.subs(y, t) # 表示t赋值给x
while abs(f1) > 0.0001:  #我们设定的精度
    #print(f1)
    dify = diff(f, y) # f对x求导
    dify = dify.subs(y, t)
    t = t - f1/dify
    f1 = f.subs(y, t)
Por1_YTM = t    



In [36]:
y = Por1_YTM + 0.00005
total_sum = 0
for number in (por1.index):
    n = por1.loc[number,"date_diff"]//180
    to_next_coupon = por1.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += por1.loc[number,"Coupon"]/2*pow((1+y),-(each + to_next_coupon/180))*number_invest
    total_sum += (100+por1.loc[number,"Coupon"]/2)*pow((1+y),-(n+to_next_coupon/180))*number_invest
    # using
new_por1_price = total_sum 
print('new Por1 price is', new_por1_price)

new Por1 price is 99533144.9848743


In [37]:
por1_PVBP = number_invest*por1["Ask Price"].sum() - new_por1_price


In [38]:
Por1_Appr_duration = por1_PVBP * (1+Por1_YTM) / new_por1_price * 10000


In [39]:
Por1_portfolio_duration = sum(por1["Ask Price"]*number_invest/fund * por1["Duration"])


In [40]:
Por1_Modified_duration = Por1_portfolio_duration / (1 + Por1_YTM)


In [41]:
Por1_YTM_plus = Por1_YTM + 0.00005
Por1_YTM_minus = Por1_YTM - 0.00005
Por1_price_plus = 0
Por1_price_minus = 0
for number in (por1.index):
    n = por1.loc[number,"date_diff"]//180
    to_next_coupon = por1.loc[number,"date_diff"]%180
    for each in range(n):
        Por1_price_plus += por1.loc[number,"Coupon"]/2*pow((1+Por1_YTM_plus),-(each + to_next_coupon/180))*number_invest
    Por1_price_plus += (100+por1.loc[number,"Coupon"]/2)*pow((1+Por1_YTM_plus),-(n+to_next_coupon/180))*number_invest
Por1_price_plus += money_left    
for number in (por1.index):
    n = por1.loc[number,"date_diff"]//180
    to_next_coupon = por1.loc[number,"date_diff"]%180
    for each in range(n):
        Por1_price_minus += por1.loc[number,"Coupon"]/2*pow((1+Por1_YTM_minus),-(each + to_next_coupon/180))*number_invest
    Por1_price_minus += (100+por1.loc[number,"Coupon"]/2)*pow((1+Por1_YTM_minus),-(n+to_next_coupon/180))*number_invest
Por1_price_minus += money_left


In [42]:
Por1_Appr_convexity = sum(por1["Ask Price"]/por1["Ask Price"].sum() * por1["Convexity"])


In [43]:
y = Por1_YTM
total_sum = 0
for number in (por1.index):
    n = por1.loc[number,"date_diff"]//180
    to_next_coupon = por1.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += por1.loc[number,"Coupon"]/2*number_invest*math.pow((1+y),-(each + to_next_coupon/180))*((each + to_next_coupon/180)+math.pow((each + to_next_coupon/180),2))
    total_sum += (100+por1.loc[number,"Coupon"]/2)*number_invest*math.pow((1+y),-(n + to_next_coupon/180))*((n + to_next_coupon/180)+math.pow((n + to_next_coupon/180),2))
total_sum = total_sum * 1/(fund*(math.pow((1+y),2)))



# Given current Macro economic conditions, you believe long term rates are going to decline in the next year. Find a portfolio(por1)(both $ exposure and number of bonds units) which is consistent with your discretionary views and your investment mandate. For por1 Calculate :

Dollar Value of the portfolio & number of each bonds units in it

Portfolio YTM

Actual and Approximate Duration

Actual Modified Duration

Actual and Approximate Convexity

In [44]:
print("Dollar value of por1 portfolio is",por1_PVBP)
print("number of por1 I will invest is",number_invest)
print("The money not used for investing is",money_left)
print('Por1 annualized YTM is', 2*t)
print("Actual por1 portfolio duration is",Por1_portfolio_duration)
print("Approximate por1 portfolio duration is",Por1_Appr_duration)
print("Actual por1 modified duration is",Por1_Modified_duration)
print("Approximate Por1 portfolio convexity is",Por1_Appr_convexity)
print("Actual annualized Por1 portfolio convexity is",total_sum/4)

Dollar value of por1 portfolio is 211650.983875692
number of por1 I will invest is 74801
The money not used for investing is 932.576388880610
Por1 annualized YTM is 0.0181203928642711
Actual por1 portfolio duration is 21.3375873094553
Approximate por1 portfolio duration is 21.4570315644216
Actual por1 modified duration is 21.1460003921484
Approximate Por1 portfolio convexity is 550.765036958363
Actual annualized Por1 portfolio convexity is 549.672973503151


In [45]:
m = sympy.symbols("m")
w1 = por1["Ask Price"].sum()/(por1["Ask Price"].sum()+m*Portfolio_price)
w2 = 1-w1

In [46]:
f = Por1_portfolio_duration * 0.5  - w1 * Por1_portfolio_duration - w2 * portfolio_duration
t = 0.0001
f1 = f.subs(m, t) # 表示t赋值给x
while abs(f1) > 0.0001:  #我们设定的精度
    #print(f1)
    dify = diff(f, m) # f对x求导
    dify = dify.subs(m, t)
    t = t - f1/dify
    f1 = f.subs(m, t)
multiplier = t    


In [47]:
# Por2 contains 1 unit of por1 and 0.13 unit of por0
# So that the duration of por2 can be half the duration of por1

In [56]:
number_por1 = math.floor(1/multiplier)
test_bond['volume'] = 1
por1["volume"] = 8
por2 = copy.deepcopy(test_bond)

In [57]:
por1_price = por1["Ask Price"].sum()
por0_price = Portfolio_price
por2_price = 8 * por1_price + por0_price

por2_accrued_price = por2_price+np.nansum(por2["Coupon"]/2*((180 - por2["date_diff"]%180)/180))

number_por2 = floor(fund / por2_accrued_price)


In [58]:
for i in por2.index:
    for j in por1.index:
        if i == j:
            por2.loc[i,"volume"] = por2.loc[i,"volume"] + por1.loc[j,"volume"]

In [59]:
y = sympy.symbols("y")
total_sum = 0
for number in (por2.index):
    n = por2.loc[number,"date_diff"]//180
    to_next_coupon = por2.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += por2.loc[number,"Coupon"]/2*pow((1+y),-(each + to_next_coupon/180))*number_por2*por2.loc[number,"volume"]
    total_sum += (100+por2.loc[number,"Coupon"]/2)*pow((1+y),-(n+to_next_coupon/180))*number_por2*por2.loc[number,"volume"]
    # using
f = (por2_price*number_por2) - total_sum
t = 0.0001
f1 = f.subs(y, t) # 表示t赋值给x
while abs(f1) > 0.0001:  #我们设定的精度
    #print(f1)
    dify = diff(f, y) # f对x求导
    dify = dify.subs(y, t)
    t = t - f1/dify
    f1 = f.subs(y, t)
Por2_YTM = t    


In [60]:
por2_duration = sum(por2["Ask Price"]*por2["volume"]/por2_price * por2["Duration"])


In [61]:
por2_MD = por2_duration/(1+t)


In [62]:
y = Por2_YTM
total_sum = 0
for number in (por2.index):
    n = por2.loc[number,"date_diff"]//180
    to_next_coupon = por2.loc[number,"date_diff"]%180
    for each in range(n):
        total_sum += por2.loc[number,"Coupon"]/2*number_por2*math.pow((1+y),-(each + to_next_coupon/180))*((each + to_next_coupon/180)+math.pow((each + to_next_coupon/180),2))
    total_sum += (100+por2.loc[number,"Coupon"]/2)*number_por2*math.pow((1+y),-(n + to_next_coupon/180))*((n + to_next_coupon/180)+math.pow((n + to_next_coupon/180),2))
total_sum = total_sum * 1/(fund*(math.pow((1+y),2)))



# Assume you constructed por1, but you decided to reduce its duration by hedging the duration risk using por0, such that the duration of new portfolio (por2) is half the duration of por1.

Find the number of unit and dollar value of por0 that you need to trade to achieve your risk objective

Find the overall composition of por2, its YTM, duration, modified duration and convexity

In [63]:
print('Multiplier of por0 is', multiplier)
print("number of por1 is",1/multiplier,"Since number of portfolio should be interger, we take 7")
print("for each unit of por0, we should buy",number_por1,"units to keep por2 duration requirement")
print("por2 price is",por2_price)
print("por2 accrued price is",por2_accrued_price)
print("number of por2 is",number_por2)

Multiplier of por0 is 0.130735763369009
number of por1 is 7.64901641471616 Since number of portfolio should be interger, we take 7
for each unit of por0, we should buy 7 units to keep por2 duration requirement
por2 price is 40062.656251299995
por2 accrued price is 40223.800348522214
number of por2 is 2486


In [64]:
por2

,Name,Ticker,Coupon,Maturity,Maturity Type,Country,Currency,Identifier,Bloomberg Composite Rating,Ask Price,Source,today,date_diff,half_YTM,YTM,Duration,MD,Convexity,volume
54,United States Treasury Note/Bond,T,1.375,2022-01-31,NORMAL,US,USD,912828Z60,NR,101.289062,BGN,2021-01-29,367,0.00377549798887384,0.00755099597774768,0.995765063958243,0.992019695592610,1.520153,1
55,United States Treasury Note/Bond,T,1.500,2022-01-31,NORMAL,US,USD,912828H86,NR,101.417969,BGN,2021-01-29,367,0.00405942198222042,0.00811884396444085,0.994896495936066,0.990874119752728,1.517756,1
56,United States Treasury Note/Bond,T,1.875,2022-01-31,NORMAL,US,USD,912828V72,NR,101.785156,BGN,2021-01-29,367,0.00500530335001219,0.0100106067000244,0.992302989371156,0.987360948308915,1.510320,1
57,United States Treasury Note/Bond,T,2.000,2022-02-15,NORMAL,US,USD,912828SF8,NR,101.984375,BGN,2021-01-29,382,0.00474579551339724,0.00949159102679449,1.03337799934632,1.02849696307342,1.614901,1
58,United States Treasury Note/Bond,T,2.500,2022-02-15,NORMAL,US,USD,9128286C9,NR,102.503906,BGN,2021-01-29,382,0.00580961677281062,0.0116192335456212,1.03016984174303,1.02421951884730,1.605183,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314,United States Treasury Note/Bond,T,2.375,2049-11-15,NORMAL,US,USD,912810SK5,NR,112.890625,BGN,2021-01-29,10517,0.00916606221887355,0.0183321244377471,21.5092167349888,21.3138526356070,564.525208,9
315,United States Treasury Note/Bond,T,2.000,2050-02-15,NORMAL,US,USD,912810SL3,NR,104.250000,BGN,2021-01-29,10609,0.00907904557557365,0.0181580911511473,22.5372380757951,22.3344624730959,602.130239,9
316,United States Treasury Note/Bond,T,1.250,2050-05-15,NORMAL,US,USD,912810SN9,NR,86.765625,BGN,2021-01-29,10698,0.00923068921338427,0.0184613784267685,24.1550127324112,23.9340846355337,675.163444,9
317,United States Treasury Note/Bond,T,1.375,2050-08-15,NORMAL,US,USD,912810SP4,NR,89.593750,BGN,2021-01-29,10790,0.00914613767301540,0.0182922753460308,24.1783524006396,23.9592180934194,674.694688,9


In [65]:
print('Por2 annualized YTM is', 2*t)
print("por2 duration is",por2_duration)
print("por2 modified duration is",por2_MD)
print("Actual annualized Por2 portfolio convexity is",total_sum/4)

Por2 annualized YTM is 0.0156509687111955
por2 duration is 10.8079671967468
por2 modified duration is 10.7240463398853
Actual annualized Por2 portfolio convexity is 78.7791260523843
